In [166]:
query = ['confidence bands for roc curves']

In [167]:
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords # preprocessing\
from math import*
from numpy import linalg as LA
import numpy as np
class Engine:
    
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        self.test_set.append(word)

    def process_score(self):
        stopWords = stopwords.words('english') 
        vectorizer = CountVectorizer()

        transformer = TfidfTransformer()

        trainVectorizerArray = vectorizer.fit_transform(self.train_set).toarray() 
        # menghitung Bobot dokumen dataset dan uji dan kemudian disimpan dalam bentuk array 
        testVectorizerArray = vectorizer.transform(self.test_set).toarray()

        cx = lambda a, b: round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 3) 
        #fungsi tanpa nama untuk normalisasi data dan definisi rumus Cosine Similarity 
        #         print testVectorizerArray
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])

        for vector in trainVectorizerArray:
            # print vector
            u = 0
            for testV in testVectorizerArray:
                #perhitungan Cosine Similarity dalam bentuk vector dari dataset dengan query
                #yang di masukan yang kemudian mengembalikan nilai cosine ke dalam variable
                #cosine_score dalam bentuk list.
                # print testV
                cosine = cx(vector, testV)
                #                 self.cosine_score.append(cosine)
                #                 bulatin = (round(cosine),2)
                output[u].append((cosine))
                u = u + 1
        return output
        # return testVectorizerArray
import pandas as pd
setD = pd.read_excel('preprocessed-dataset.xlsx',sep=',')
#print(setD)
Doc = setD['preprocessed_judul'].astype('str')
#print(Doc)
# kunci = pd.read_excel('query.xlsx',sep=',')
# query = kunci['judul'].astype('str')


engine = Engine()

docs = [str(x) for x in Doc]
documentNames = list()

for i, doc in enumerate(docs):
    engine.addDocument(doc) 
    documentNames.append("Document_{}".format(i+1))
for queries in query:
    engine.setQuery(queries) #inputandata uji

titles_score = engine.process_score()
ScoreDf = (pd.DataFrame(titles_score)).T
ScoreDf.columns = query
ScoreDf["Documents"] = documentNames
ScoreDf["Abstrak"] = setD["preprocessed_abstrak"].values
ScoreDf

from sklearn.model_selection import cross_val_score
import time

#svm_ = svm.SVC(kernel='linear')
df_listed = list()
for i in query:
    labels = list()
    #labelss = list()
    for j in ScoreDf[i]:
        if j>0.000:
            labels.append(1)
            #labelss.append(cross_val_score(svm_, query, Doc, cv=10)
        else:
            labels.append(0)
            #labelss.append(cross_val_score(svm_, query, Doc, cv=10)
    datadf = pd.DataFrame(ScoreDf[i])
    datadf['Documents'] = ScoreDf['Documents']
    datadf['Labels'] = labels
    datadf['abstrak'] = ScoreDf['Abstrak'].values
    datadf['reviewer'] = setD['Reviewer'].values
    datadf['Judul'] = setD['Judul'].values
    df_listed.append(datadf.sort_values(by=[i], ascending=False))
#df_listed
#df_listed[0].head(5)

#import urllib2  
import time  
awal = time.time()
df_listed[0].head(10) 
akhir = time.time()  
print ("Total Waktu Proses ", akhir- awal, " Detik.") 

Total Waktu Proses  0.0009987354278564453  Detik.


In [168]:
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords # preprocessing\
import math
from numpy import linalg as LA
import numpy as np
class Engine:
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        self.test_set.append(word)

    def process_score(self):
        vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True)
        svd_model = TruncatedSVD(n_components=100,algorithm='randomized',n_iter=10)
        lsa = Pipeline([('tfidf', vectorizer),('svd', svd_model)])
        #print(lsa)
        #transformer = TfidfTransformer(stop_words='english', use_idf=True, smooth_idf=True)

        trainVectorizerArray = lsa.fit_transform(self.train_set).tolist() 
        # menghitung Bobot dokumen dataset dan uji dan kemudian disimpan dalam bentuk array 
        testVectorizerArray = lsa.transform(self.test_set).tolist()

        cx = lambda a, b: round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 3) 
        #fungsi tanpa nama untuk normalisasi data dan definisi rumus Cosine Similarity 
        #print (testVectorizerArray)
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])

        for vector in trainVectorizerArray:
            #print (vector)
            u = 0
            for testV in testVectorizerArray:
                #perhitungan Cosine Similarity dalam bentuk vector dari dataset dengan query
                #yang di masukan yang kemudian mengembalikan nilai cosine ke dalam variable
                #cosine_score dalam bentuk list.
                #print (testV)
                cosine = cx(vector, testV)
                #print(cosine)
                #                 self.cosine_score.append(cosine)
                #                 bulatin = (round(cosine),2)
                if math.isnan(cosine):
                    cosine = 0
                output[u].append((cosine))
                u = u + 1
        return output
        #return testVectorizerArray
        
import pandas as pd
setD = pd.read_excel('preprocessed-dataset.xlsx',sep=',')
#print(setD)
Doc = setD['preprocessed_judul'].astype('str')
#print(Doc)
# kunci = pd.read_excel('query.xlsx',sep=',')
# query = kunci['judul'].astype('str')

        
engine = Engine()

docs = [str(x) for x in Doc]
documentNames = list()

for i, doc in enumerate(docs):
    engine.addDocument(doc) 
    documentNames.append("Document_{}".format(i+1))
for queries in query:
    engine.setQuery(queries) #inputandata uji

titles_score = engine.process_score()
ScoreDf = (pd.DataFrame(titles_score)).T
ScoreDf.columns = query
ScoreDf["Documents"] = documentNames
ScoreDf["Abstrak"] = setD["preprocessed_abstrak"].values
ScoreDf

from sklearn.model_selection import cross_val_score
import time

#svm_ = svm.SVC(kernel='linear')
df_listed = list()
for i in query:
    labels = list()
    #labelss = list()
    for j in ScoreDf[i]:
        if j>0.000:
            labels.append(1)
            #labelss.append(cross_val_score(svm_, query, Doc, cv=10)
        else:
            labels.append(0)
            #labelss.append(cross_val_score(svm_, query, Doc, cv=10)
    datadf = pd.DataFrame(ScoreDf[i])
    datadf['Documents'] = ScoreDf['Documents']
    datadf['Labels'] = labels
    datadf['abstrak'] = ScoreDf['Abstrak'].values
    datadf['reviewer'] = setD['Reviewer'].values
    datadf['Judul'] = setD['Judul'].values
    df_listed.append(datadf.sort_values(by=[i], ascending=False))
#df_listed
#df_listed[0].head(5)
#import urllib2  
import time  
awal = time.time()
df_listed[0].head(10) 
akhir = time.time()  
print ("Total Waktu Proses ", akhir- awal, " Detik.") 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


Total Waktu Proses  0.0009980201721191406  Detik.
